In [12]:
from subprocess import Popen, PIPE
import numpy as np
import json
import torch
import matplotlib.pyplot as plt

In [13]:
p = Popen(['../cpp_backend/cmake-build-debug/cpp_backend'], shell=True, stdout=PIPE, stdin=PIPE)

def send(p, s):
    p.stdin.write(s)
    p.stdin.flush()
    print("sent:", s)

    line = p.stdout.readline()
    result = line.decode("utf-8") 
    print("recieved:", result)

    return result 

for i in range(100):
    a_id = 0
    a1 = np.random.randint(20)
    a2 = np.random.randint(20)
    
    def get_q(): return '{"action_id":'+str(a_id)+',"params":[' + str(a1) + ',' + str(a2) + ']}\n' 
    q = get_q()
    q = str.encode(q)
    
    send(p, q)
    
line = p.stdout.readline()
result = line.decode("utf-8") 
print(result)

p.kill()

sent: b'{"action_id":0,"params":[14,10]}\n'
recieved: 


BrokenPipeError: [Errno 32] Broken pipe

In [17]:
from torch_geometric.data import Data

with open('../cpp_backend/cmake-build-debug/graph.json') as f:
    j = json.loads(''.join(f.readlines()))

def construct_data(j):
    # Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])
    
    num_edges = len(j['e'])
    num_vertices = len(j['v'])
    
    d = Data() 
    d.edge_index = torch.zeros(2, num_edges).long()
    d.train_mask = torch.zeros(num_vertices).bool()
    d.test_mask = torch.zeros(num_vertices).bool()
    d.val_mask = torch.zeros(num_vertices).bool()
    d.edge_attr = torch.zeros(num_edges, 1).float()
    
    d.x = torch.zeros(num_vertices, 1).float()
    d.y = torch.zeros(num_vertices).long()
    
    
    for i, (id, tag, _) in enumerate(j['v']):
        d.x[id] = 0
        d.y[id] = tag
        
        d.train_mask[id] = (id % 3) == 0 
        d.val_mask[id] = (id % 3) == 1 
        d.test_mask[id] = (id % 3) == 2
        
    for i, (v, u, tag) in enumerate(j['e']):
        d.edge_index[0][i] = v
        d.edge_index[1][i] = u
        d.edge_attr[i][0] = tag
        
    return d

d = construct_data(j)
print(d)

Data(edge_attr=[1444, 1], edge_index=[2, 1444], test_mask=[570], train_mask=[570], val_mask=[570], x=[570, 1], y=[570])


In [20]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, MessagePassing
from cg_conv import CGConv

data = construct_data(j)

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # self.conv1 = GATConv(1, 2, heads=1, dropout=0.0)
        
        self.lin1 = torch.nn.Linear(1, 4)
        
        self.conv1 = CGConv(channels=4, dim=1)
        self.conv2 = CGConv(channels=4, dim=1)
        
        # self.conv2 = GATConv(1, 4, heads=1, concat=True, dropout=0.6)
        self.lin2 = torch.nn.Linear(4, 2)
        
        # self.conv2 = GATConv(1, 4, heads=1, concat=True, dropout=0.6)

    def forward(self):
        x = data.x
        # x = F.dropout(data.x, p=0.2, training=self.training)
        # x = F.leaky_relu(self.conv1(x, data.edge_index, data.edge_attr))
        # x = self.lin(x) 
        x = self.lin1(x)
        x = self.conv1(x, data.edge_index, data.edge_attr)
        x = self.conv2(x, data.edge_index, data.edge_attr)
        x = self.lin2(x)
        # x = F.dropout(x, p=0.6, training=self.training)
        # x = self.conv2(x, data.edge_index)
        return F.log_softmax(x, dim=1)


device = torch.device('cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=4e-3, weight_decay=1e-5)

def train():
    model.train()
    optimizer.zero_grad()
    loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.detach().numpy()


def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        # print(logits)
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


for epoch in range(1, 5000):
    loss = train()
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    if epoch % 5 == 0:
        print(log.format(epoch, *test()) + ' Loss: {:.4f}'.format(loss))

Epoch: 005, Train: 0.8105, Val: 0.7579, Test: 0.8105 Loss: 0.5589
Epoch: 010, Train: 0.8105, Val: 0.7632, Test: 0.8158 Loss: 0.4891
Epoch: 015, Train: 0.8105, Val: 0.7632, Test: 0.8158 Loss: 0.4835
Epoch: 020, Train: 0.8105, Val: 0.7632, Test: 0.8158 Loss: 0.4716
Epoch: 025, Train: 0.8105, Val: 0.7632, Test: 0.8158 Loss: 0.4528
Epoch: 030, Train: 0.8105, Val: 0.7632, Test: 0.8158 Loss: 0.4313
Epoch: 035, Train: 0.8105, Val: 0.7684, Test: 0.8158 Loss: 0.4075
Epoch: 040, Train: 0.8474, Val: 0.8158, Test: 0.8368 Loss: 0.3814
Epoch: 045, Train: 0.8579, Val: 0.8316, Test: 0.8421 Loss: 0.3567
Epoch: 050, Train: 0.8526, Val: 0.8368, Test: 0.8368 Loss: 0.3330
Epoch: 055, Train: 0.8684, Val: 0.8421, Test: 0.8474 Loss: 0.3146
Epoch: 060, Train: 0.8737, Val: 0.8474, Test: 0.8474 Loss: 0.3032
Epoch: 065, Train: 0.8737, Val: 0.8579, Test: 0.8684 Loss: 0.2936
Epoch: 070, Train: 0.8789, Val: 0.8579, Test: 0.8684 Loss: 0.2841
Epoch: 075, Train: 0.8895, Val: 0.8632, Test: 0.8737 Loss: 0.2741
Epoch: 080

KeyboardInterrupt: 